<a href="https://colab.research.google.com/github/surajsrivathsa/deep_learning_paper_implementations/blob/master/Bio_Inspired_Bistable_Recurrent_Cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bio Inspired Recurrent Cell
**For Long lasting memory**

paper link: https://arxiv.org/pdf/2006.05252

paper discussion link: https://www.youtube.com/watch?v=DLq1DUcMh1Q

Credits: Yannic Kilcher (https://www.bitchute.com/video/TLuAfQz5SlKF/ )



# **Tasks**

1. Code in a BRC, NBRC and GRU cells

2. Compare BRC vs NBRC vs GRU for a simple dataset

3. Make notes, Plot graphs

4. Analyze advantages and disadvantages of BRC/NBRC

In [1]:
import os
%tensorflow_version 2.x
import tensorflow as tf
from sklearn import preprocessing
from google.colab import files
from google.colab import drive
import matplotlib.pyplot as plt
import pandas as pd
import copy
import numpy as np

In [2]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import initializers
import tensorboard
import time
from datetime import datetime
from keras import backend as K
from prepare_data import parse_seq
import pickle

Using TensorFlow backend.


In [3]:
 print(os.getcwd())
print(tf.__version__)

/content
2.2.0


In [4]:
path = '.'
 
files = os.listdir(path)
for name in files:
    print(name)

.config
skp.tfrecords
prepare_data.py
skp_vocab
__pycache__
sample_data


# Data Preprocessing

In [6]:

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x, 200))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{'\n': 1, 'C': 2, 'l': 3, '!': 4, '?': 5, 'q': 6, 'f': 7, 't': 8, 'J': 9, 'n': 10, ':': 11, 'i': 12, 'r': 13, 'R': 14, 's': 15, ' ': 16, 'v': 17, 'I': 18, 'V': 19, '-': 20, 'Z': 21, 'X': 22, 'z': 23, 'e': 24, 'U': 25, 'F': 26, 'N': 27, '[': 28, 'm': 29, 'S': 30, 'd': 31, 'O': 32, '3': 33, 'a': 34, ']': 35, 'W': 36, 'c': 37, 'k': 38, 'Y': 39, 'L': 40, 'P': 41, 'w': 42, 'B': 43, "'": 44, 'E': 45, 'x': 46, 'H': 47, 'M': 48, 'u': 49, ',': 50, 'y': 51, 'h': 52, 'Q': 53, '&': 54, 'G': 55, 'b': 56, 'D': 57, '$': 58, 'K': 59, 'T': 60, 'o': 61, '.': 62, 'g': 63, 'j': 64, ';': 65, 'A': 66, 'p': 67, '<S>': 0}
68


In [7]:
def onehotencode(ds):
  
  new_data = tf.one_hot(indices = ds, depth = vocab_size)
  return new_data;

new_data = data.map(onehotencode)

In [8]:
cnt = 0
counter = 0
for element in data:
  counter = counter + 1
  print(element)
  if(counter > 5):
    break;

tf.Tensor(
[ 0 26 12 13 15  8 16  2 12  8 12 23 24 10 11  1 43 24  7 61 13 24 16 42
 24 16 67 13 61 37 24 24 31 16 34 10 51 16  7 49 13  8 52 24 13 50 16 52
 24 34 13 16 29 24 16 15 67 24 34 38 62  1  1 66  3  3 11  1 30 67 24 34
 38 50 16 15 67 24 34 38 62  1  1 26 12 13 15  8 16  2 12  8 12 23 24 10
 11  1 39 61 49 16 34 13 24 16 34  3  3 16 13 24 15 61  3 17 24 31 16 13
 34  8 52 24 13 16  8 61 16 31 12 24 16  8 52 34 10 16  8 61 16  7 34 29
 12 15 52  5  1  1 66  3  3 11  1 14 24 15 61  3 17 24 31 62 16 13 24 15
 61  3 17 24 31 62  1  1 26 12 13 15  8 16  2 12  8 12 23 24 10 11  1 26
 12 13 15  8 50 16 51 61], shape=(200,), dtype=int32)
tf.Tensor(
[ 0  8 16  2 12  8 12 23 24 10 11  1 26 12 13 15  8 50 16 51 61 49 16 38
 10 61 42 16  2 34 12 49 15 16 48 34 13 37 12 49 15 16 12 15 16 37 52 12
 24  7 16 24 10 24 29 51 16  8 61 16  8 52 24 16 67 24 61 67  3 24 62  1
  1 66  3  3 11  1 36 24 16 38 10 61 42 44  8 50 16 42 24 16 38 10 61 42
 44  8 62  1  1 26 12 13 15  8 16  2 12  8 12 23

In [ ]:
cnt = 0
for element in data:
  cnt = cnt+1
print(cnt)

25549


# GRU Cell

In [9]:
def initialize_parameters(n_h, vocab_size):
  parameter_dict = {}

  parameter_dict["W_h"] = tf.Variable(tf.initializers.GlorotUniform(seed = 0)(shape=[n_h, n_h + vocab_size]))
  print("W_h: " + str(parameter_dict["W_h"].shape))
  parameter_dict["b_h"] = tf.Variable(tf.initializers.GlorotUniform(seed = 2)(shape=[ n_h, 1]))
  print("b_h: " + str(parameter_dict["b_h"].shape))


  parameter_dict["W_u"] = tf.Variable(tf.initializers.GlorotUniform(seed = 3)(shape=[n_h, n_h + vocab_size]))
  print("W_u: " + str(parameter_dict["W_u"].shape))
  parameter_dict["b_u"] = tf.Variable(tf.initializers.GlorotUniform(seed = 2)(shape=[ n_h, 1]))
  print("b_u: " + str(parameter_dict["b_u"].shape))

  parameter_dict["W_r"] = tf.Variable(tf.initializers.GlorotUniform(seed = 4)(shape=[n_h, n_h + vocab_size]))
  print("W_r: " + str(parameter_dict["W_r"].shape))
  parameter_dict["b_r"] = tf.Variable(tf.initializers.GlorotUniform(seed = 5)(shape=[ n_h, 1]))
  print("b_r: " + str(parameter_dict["b_r"].shape))

  parameter_dict["W_o"] = tf.Variable(tf.initializers.GlorotUniform(seed = 7)(shape=[n_h, vocab_size]))
  print("W_o: " + str(parameter_dict["W_o"].shape))
  parameter_dict["b_o"] = tf.Variable(tf.initializers.GlorotUniform(seed = 6)(shape=[ 1, vocab_size]))
  print("b_o: " + str(parameter_dict["b_o"].shape))
  
  return parameter_dict;

In [10]:
ones_tensor = tf.Variable(tf.ones(shape =[512, 128], dtype=tf.dtypes.float32))

def gru_forward_compute(x_t, parameter_dict, h_prev):

  stacked_input = tf.concat([h_prev, tf.transpose(x_t)], axis = 0)
  
  update_gate = tf.nn.sigmoid(tf.matmul(parameter_dict["W_u"], stacked_input ) + parameter_dict["b_u"])
  #print(update_gate.shape)

  reset_gate = tf.nn.sigmoid(tf.matmul(parameter_dict["W_r"], stacked_input ) + parameter_dict["b_r"])
  #print(reset_gate.shape)

  hadamard_product = tf.multiply(reset_gate, h_prev)
  stacked_cell_input = tf.concat([hadamard_product, tf.transpose(x_t)], axis = 0)
  h_tilde = tf.nn.tanh(tf.matmul(parameter_dict["W_h"], stacked_cell_input ) + parameter_dict["b_h"])

  
  h = tf.multiply(update_gate, h_tilde) + tf.multiply(tf.subtract(ones_tensor, update_gate), h_prev)

  logits = tf.matmul(tf.transpose(h), parameter_dict["W_o"] ) + parameter_dict["b_o"]
  softmax_output = tf.nn.softmax(logits,axis = 0)

  return [update_gate, reset_gate, h, logits, softmax_output];

In [11]:
parameter_dict = initialize_parameters(n_h=512, vocab_size=vocab_size)

W_h: (512, 580)
b_h: (512, 1)
W_u: (512, 580)
b_u: (512, 1)
W_r: (512, 580)
b_r: (512, 1)
W_o: (512, 68)
b_o: (1, 68)


In [12]:
x_t = tf.Variable(tf.initializers.GlorotUniform(seed = 0)(shape=[128, vocab_size]))
h_prev = tf.Variable(tf.initializers.GlorotUniform(seed = 0)(shape=[512, 128]))
update_gate, reset_gate, h, logits, softmax_output = gru_forward_compute(x_t, parameter_dict, h_prev)

In [13]:
tf.shape(softmax_output)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([128,  68], dtype=int32)>

In [14]:
#print(update_gate)
#print(h)
print(tf.reduce_sum(softmax_output[:, 2]))
print(softmax_output[:, 2])

tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(
[0.00768709 0.00754427 0.00781152 0.00689983 0.00797018 0.00783288
 0.00819303 0.00695316 0.00779048 0.00770025 0.0077992  0.007536
 0.00870549 0.00717369 0.00744747 0.00825853 0.00802295 0.00742147
 0.00707195 0.00743077 0.00827387 0.00805128 0.00808253 0.00789529
 0.00762278 0.00769768 0.00783361 0.00785466 0.00769155 0.00829025
 0.0080389  0.00824371 0.00803453 0.007808   0.00812774 0.00790186
 0.00803084 0.00734907 0.0072067  0.00739729 0.00877008 0.00763129
 0.00824674 0.00768775 0.00723501 0.00760793 0.0082252  0.00833443
 0.0079906  0.00775657 0.00799972 0.00757066 0.00751612 0.00721304
 0.00726312 0.00730189 0.00755075 0.00829403 0.00777673 0.007787
 0.00768325 0.0081652  0.00750713 0.00760562 0.00800111 0.00778991
 0.00821048 0.00813209 0.00767239 0.00842456 0.00790222 0.00751437
 0.00789996 0.00755009 0.00721819 0.00832501 0.00765466 0.008571
 0.00843915 0.00767957 0.0080774  0.00801731 0.00788372 0.00759925
 0.00851206 0.007

# BRC Cell

In [79]:
n_h = 512
batch_size = 128

def initialize_brc_parameters(n_h, batch_size, vocab_size):
  brc_parameter_dict = {}

  brc_parameter_dict["W_c"] = tf.Variable(initial_value = tf.keras.initializers.glorot_uniform(seed = 1)(shape = (n_h, batch_size)))
  brc_parameter_dict["W_a"] = tf.Variable(initial_value = tf.keras.initializers.glorot_uniform(seed = 2)(shape = (n_h, batch_size)))
  brc_parameter_dict["U_c"] = tf.Variable(initial_value = tf.keras.initializers.glorot_uniform(seed = 3)(shape = (n_h, vocab_size)))
  brc_parameter_dict["U_a"] = tf.Variable(initial_value = tf.keras.initializers.glorot_uniform(seed = 4)(shape = (n_h, vocab_size)))
  brc_parameter_dict["U_x"] = tf.Variable(initial_value = tf.keras.initializers.glorot_uniform(seed = 4)(shape = (n_h, vocab_size)))
  brc_parameter_dict["b_a"] = tf.Variable(tf.initializers.GlorotUniform(seed = 5)(shape=[ n_h, 1]))
  brc_parameter_dict["b_c"] = tf.Variable(tf.initializers.GlorotUniform(seed = 6)(shape=[ n_h, 1]))
  brc_parameter_dict["W_o"] = tf.Variable(tf.initializers.glorot_uniform(seed = 7)(shape=[n_h, vocab_size]))
  brc_parameter_dict["b_o"] = tf.Variable(tf.initializers.glorot_uniform(seed = 8)(shape=[1, vocab_size]))

  return brc_parameter_dict;

In [80]:
brc_parameter_dict = initialize_brc_parameters(n_h = 512, batch_size=128, vocab_size=vocab_size)
for param_name, param_val in brc_parameter_dict.items():
  print("param_name: {} | shape: {}".format(param_name, str(param_val.shape)))

param_name: W_c | shape: (512, 128)
param_name: W_a | shape: (512, 128)
param_name: U_c | shape: (512, 68)
param_name: U_a | shape: (512, 68)
param_name: U_x | shape: (512, 68)
param_name: b_a | shape: (512, 1)
param_name: b_c | shape: (512, 1)
param_name: W_o | shape: (512, 68)
param_name: b_o | shape: (1, 68)


BRC forward compute

In [81]:
@tf.function
def brc_forward_compute(x_t, parameter_dict, h_prev, brc_ones_tensor):

  #update gate
  tmp_at = tf.add(tf.matmul(a = brc_parameter_dict["U_a"], b = x_t, transpose_b= True) , tf.multiply(brc_parameter_dict["W_a"], h_prev))
  A_t = tf.add(brc_ones_tensor, tf.keras.activations.tanh(tmp_at))

  #Reset gate
  tmp_ct = tf.add(tf.matmul(a = brc_parameter_dict["U_c"],b = x_t, transpose_b = True) , tf.multiply(brc_parameter_dict["W_c"], h_prev))
  C_t = tf.keras.activations.sigmoid(tmp_ct)

  #Current cell value
  tmp_current_val = tf.add(tf.matmul(a=brc_parameter_dict["U_x"], b=x_t, transpose_b=True) , tf.multiply(A_t, h_prev))
  H_t = tf.add(tf.multiply(C_t, h_prev) , tf.multiply((1-C_t), tf.keras.activations.tanh(tmp_current_val)))

  #Output of cell
  logits = tf.matmul(tf.transpose(H_t), parameter_dict["W_o"] ) + parameter_dict["b_o"]
  softmax_output = tf.nn.softmax(logits,axis = 0)

  return [A_t, C_t, H_t, logits, softmax_output];



Sanity Check on sample data

In [82]:
x_t = tf.Variable(tf.initializers.GlorotUniform(seed = 0)(shape=[128, vocab_size]))
h_prev = tf.Variable(tf.initializers.GlorotUniform(seed = 0)(shape=[512, 128]))

In [70]:
A_t, C_t, H_t, logits, softmax_output = brc_forward_compute(x_t, parameter_dict, h_prev)

TypeError: ignored

In [52]:
softmax_output[1,:]

<tf.Tensor: shape=(68,), dtype=float32, numpy=
array([0.00803658, 0.00850225, 0.00673646, 0.00723308, 0.00839005,
       0.00740324, 0.00682028, 0.00783177, 0.0078986 , 0.00750549,
       0.0082809 , 0.00847614, 0.00735491, 0.00779753, 0.00873234,
       0.00849496, 0.00663494, 0.00790718, 0.00792656, 0.0090681 ,
       0.00705698, 0.00754907, 0.00646358, 0.00783225, 0.00817844,
       0.00661802, 0.00695915, 0.00730259, 0.0083609 , 0.00756912,
       0.00746488, 0.00724498, 0.00815309, 0.00829601, 0.00745946,
       0.00778952, 0.0078167 , 0.00747664, 0.00710692, 0.00745986,
       0.00799013, 0.00698474, 0.00657904, 0.00723013, 0.00755678,
       0.00756149, 0.007815  , 0.00919298, 0.00783879, 0.00755976,
       0.00807544, 0.00828158, 0.00854651, 0.00739413, 0.00763806,
       0.00797055, 0.00790042, 0.00809043, 0.00906261, 0.00804481,
       0.00767838, 0.00786132, 0.00682665, 0.00720295, 0.00783846,
       0.00962132, 0.00705957, 0.00698116], dtype=float32)>

In [83]:
@tf.function
def compute_loss_per_time_step(lbl_batch, logits):
  #print(lbl_batch.shape)
  #print(logits.shape)
  loss_per_time_step = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = lbl_batch, logits = logits))
  return loss_per_time_step;

In [85]:
@tf.function
def brc_one_train_step(x_batch_train, val_data, h_prev, time_steps, optimizer, train_acc_metric, 
                       parameter_dict, logits, out_one_time_step, brc_ones_tensor):

  trainable_parameter_names = []
  trainable_parameter_values = []
  for key, val in parameter_dict.items():
    trainable_parameter_names.append(key)
    trainable_parameter_values.append(val)
  
  loss_per_batch = tf.TensorArray(tf.float32, size=200)
  with tf.GradientTape() as tape:
    for time_step in tf.range(time_steps-1):
          
      x_t = x_batch_train[:, time_step, :]
      #print("shape of each time slice: {} ".format(x_t.shape))
      lbl_batch = val_data[:, time_step+1]
      #print("labels shape: {}".format(lbl_batch.shape))
      A_t, C_t, H_t, logits, softmax_output = brc_forward_compute(x_t, parameter_dict, h_prev, brc_ones_tensor)
      loss_per_time_step = compute_loss_per_time_step(lbl_batch = lbl_batch, logits=logits)
      # loss_per_batch.append(loss_per_time_step)
      h_prev = H_t

      loss_per_batch = loss_per_batch.write(time_step, loss_per_time_step)
      train_acc_metric(lbl_batch, logits)

    avg_loss = tf.math.reduce_mean(loss_per_batch.stack())
  # # # weight, bias vector updates

  grads = tape.gradient(avg_loss, trainable_parameter_values)

  optimizer.apply_gradients(zip(grads, trainable_parameter_values))
  #grads = tape.gradient(avg_loss, (w_hh1, w_xh1, b_h1, w_ho1, b_o1))
  #opt.apply_gradients(zip(grads, (w_hh1, w_xh1, b_h1, w_ho1, b_o1)))
  # print("Average loss: {}".format(avg_error))
  train_acc = train_acc_metric.result()
  return logits, out_one_time_step, h_prev, avg_loss, train_acc

In [86]:
# create batches of data
# setting drop remainder to true would omit 25549 % 128 = 77 examples from training
# but this is important as batchsize is used as one of the dimensions inside weight matrices and weight dimensions must not change on the same data
batched_vocab_data = data.batch(batch_size=128,drop_remainder=True)
batched_onehotencoded_data = new_data.batch(batch_size=128, drop_remainder=True)
# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=   1 * 1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

#initilaize parametrrs and epochs
epochs = 15
parameter_dict = initialize_brc_parameters(512, batch_size, vocab_size)
brc_ones_tensor = tf.Variable(initial_value=tf.ones(shape = (n_h, batch_size)))

In [87]:
def BRCModel(epochs,onehotencoded_train_data, vocab_test_data, parameter_dict, vocab_size = vocab_size, time_steps = 200, brc_ones_tensor = brc_ones_tensor ):
  train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
  optimizer = tf.keras.optimizers.Adam(learning_rate = 1.5 * 1e-3)
  h_prev = tf.zeros((512,128), dtype=tf.dtypes.float32)
  logits = tf.Variable(tf.zeros((128,vocab_size), dtype=tf.dtypes.float32))
  out_one_time_step  = tf.Variable(tf.zeros((128,vocab_size), dtype=tf.dtypes.float32))
  for e in tf.range(epochs):
    start = time.time()
    for batch_num, (x_batch_train, val_data) in enumerate(zip(onehotencoded_train_data,vocab_test_data) ):

      logits, out_one_time_step, h_prev, avg_loss, train_acc = brc_one_train_step(x_batch_train, val_data, h_prev, time_steps, optimizer, train_acc_metric, parameter_dict, logits, out_one_time_step, brc_ones_tensor)
    
    # Display metrics at the end of each epoch.
    print("Epoch: {}/{}, Loss: {} Accuracy: {}".format(e+1,epochs, avg_loss, (train_acc.numpy()*100)))
    train_acc_metric.reset_states()
    stop = time.time()
    print("took {} seconds\n".format(stop-start))

In [88]:
BRCModel(epochs,batched_onehotencoded_data, batched_vocab_data, parameter_dict )

Epoch: 1/15, Loss: 3.1275856494903564 Accuracy: 14.696283638477325
took 86.58064579963684 seconds

Epoch: 2/15, Loss: 3.076092004776001 Accuracy: 16.12396091222763
took 83.32192540168762 seconds

Epoch: 3/15, Loss: 3.043930768966675 Accuracy: 16.927820444107056
took 83.89748167991638 seconds

Epoch: 4/15, Loss: 3.0209500789642334 Accuracy: 17.506995797157288
took 84.85212135314941 seconds

Epoch: 5/15, Loss: 3.0033252239227295 Accuracy: 17.942985892295837
took 84.67544150352478 seconds

Epoch: 6/15, Loss: 2.9893276691436768 Accuracy: 18.27520579099655
took 86.02694249153137 seconds

Epoch: 7/15, Loss: 2.977917432785034 Accuracy: 18.529006838798523
took 85.73701024055481 seconds

Epoch: 8/15, Loss: 2.968404531478882 Accuracy: 18.74193102121353
took 86.31261897087097 seconds

Epoch: 9/15, Loss: 2.9603164196014404 Accuracy: 18.922561407089233
took 84.81904935836792 seconds

Epoch: 10/15, Loss: 2.953324794769287 Accuracy: 19.077308475971222
took 84.9548728466034 seconds

Epoch: 11/15, Loss

param_name: W_c | shape: (512, 128)

param_name: W_a | shape: (512, 128)

param_name: U_c | shape: (512, 68)

param_name: U_a | shape: (512, 68)

param_name: U_x | shape: (512, 68)

param_name: b_a | shape: (512, 1)

param_name: b_c | shape: (512, 1)

param_name: W_o | shape: (512, 68)

param_name: b_o | shape: (1, 68)

In [89]:
import pickle

filename = "brc_15epochs.pickle"
with open(filename, 'wb') as f:
  parameter_list = [brc_parameter_dict["W_c"], brc_parameter_dict["W_a"], brc_parameter_dict["U_c"], brc_parameter_dict["U_a"], 
                    brc_parameter_dict["U_x"], brc_parameter_dict["b_a"] ,brc_parameter_dict["b_c"], brc_parameter_dict["W_o"], 
                    brc_parameter_dict["b_o"], h_prev]
  pickle.dump(parameter_list, f)